## Wikipedia Scrapping

In [1]:
import pandas as pd

In [23]:
#read the wiki page
toronto_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [24]:
#drop rows where Borough is not assigned
toronto_df = toronto_df.drop(toronto_df[toronto_df['Borough']=='Not assigned'].index)

In [29]:
toronto_df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [27]:
#no postal code duplicates
toronto_df['Postal code'].unique().shape

(103,)

In [28]:
toronto_df.shape

(103, 3)

## Adding Lat/ Lon

In [8]:
csv = pd.read_csv('https://cocl.us/Geospatial_data')

In [32]:
csv.rename(columns = {'Postal Code':'Postal code'}, inplace = True)

In [33]:
csv.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
#add longitude
toronto_df = pd.merge(toronto_df, csv[['Postal code', 'Longitude']], on = 'Postal code', how = 'left')

In [37]:
#add latitude
toronto_df = pd.merge(toronto_df, csv[['Postal code', 'Latitude']], on = 'Postal code', how = 'left')

In [40]:
toronto_df[toronto_df['Neighborhood']=='Leaside']

,Postal code,Borough,Neighborhood,Longitude,Latitude
23,M4G,East York,Leaside,-79.363452,43.70906


In [41]:
toronto_df.head(10)

,Postal code,Borough,Neighborhood,Longitude,Latitude
0,M3A,North York,Parkwoods,-79.329656,43.753259
1,M4A,North York,Victoria Village,-79.315572,43.725882
2,M5A,Downtown Toronto,Regent Park / Harbourfront,-79.360636,43.654260
3,M6A,North York,Lawrence Manor / Lawrence Heights,-79.464763,43.718518
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,-79.389494,43.662301
5,M9A,Etobicoke,Islington Avenue,-79.532242,43.667856
6,M1B,Scarborough,Malvern / Rouge,-79.194353,43.806686
7,M3B,North York,Don Mills,-79.352188,43.745906
8,M4B,East York,Parkview Hill / Woodbine Gardens,-79.309937,43.706397
9,M5B,Downtown Toronto,"Garden District, Ryerson",-79.378937,43.657162


## Start Clustering Downtown

In [ ]:
downtown_df = toronto_df[toronto_df['Borough']=='Downtown Toronto']

In [ ]:
downtown_df

In [51]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


branca-0.4.0         | 26 KB     | ##################################### | 1

In [54]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[downtown_df['Latitude'].mean(), downtown_df['Longitude'].mean()], zoom_start=14)

# add markers to map
for lat, lng, borough, neighborhood in zip(downtown_df['Latitude'], downtown_df['Longitude'], downtown_df['Borough'], downtown_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto